# import packages

In [1]:
import pandas as pd
import scipy.stats as ss
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'
import scipy
scipy.__version__

'1.9.2'

# read rpkm calculated by stringtie

In [41]:
rpkm=pd.read_csv("/home/user/data2/lit/project/ZNF271/02-APA/analysis/stringtie_whole_genome/stringtie.rpkm.txt",sep='\t',header=None)

# read metadata,fragmentation score,period

In [42]:
metadata=pd.read_csv("/home/user/data/lit/project/ZNF271/data/rna-seq/brain/metadata.txt",sep='\t')
frag_score=pd.read_csv("/home/user/data/lit/project/ZNF271/data/rna-seq/brain/frag_score.txt",sep='\t')
period=pd.read_csv("/home/user/data/lit/project/ZNF271/data/rna-seq/brain/period.txt",sep='\t')
metadata.head()
frag_score.head()
period.head()

,sample,Sex,Developmental_Stage,Age,Unit,organism
0,1988sTS.Human.Testis.10w.Male,male,10 pcw,10,week,testis
1,2024sTSb.Human.Heart.10w.Male,male,10 pcw,10,week,heart
2,2025sTSb.Human.Liver.10w.Male,male,10 pcw,10,week,liver
3,2092sTS.Human.Heart.10w.Female,female,10 pcw,10,week,heart
4,2094sTS.Human.Liver.10w.Female,female,10 pcw,10,week,liver


,sample,frag_score
0,2026sTSm.Human.Forebrain.CS21.Male,0.879879
1,2029sTSb.Human.Hindbrain.CS21.Male,0.860425
2,2031sTS.Human.Hindbrain.CS22.Male,0.860896
3,2033sTS.Human.Hindbrain.CS22.Female,0.890121
4,2034sTS.Human.Forebrain.CS22.Female,0.883731


,Developmental_Stage,period,period.abbre,period.abbre.abbre
0,4 pcw,Embryonic,Embryonic,Before birth
1,5 pcw,Embryonic,Embryonic,Before birth
2,6 pcw,Embryonic,Embryonic,Before birth
3,7 pcw,Embryonic,Embryonic,Before birth
4,8 pcw,Early fetal,Fetal,Before birth


# read gene list

In [43]:
t_exon_g_l=pd.read_csv("/home/user/data2/lit/project/ZNF271/02-APA/terminal_exon.gene.lst",sep='\t',header=None)
t_exon_g_l.columns=['gene_id']
len(t_exon_g_l)

6457

# rpkm_proximal and rpkm_distal

In [44]:
rpkm.columns=["type","gene_id","rpkm","sample"]
rpkm_pro=rpkm.query("type.str.contains('proximal')")
rpkm_pro.columns=["type","gene_id","rpkm_pro","sample"]
rpkm_dis=rpkm.query("type.str.contains('distal')")
rpkm_dis.columns=["type","gene_id","rpkm_dis","sample"]
len(rpkm_pro)
len(rpkm_dis)


736098

736098

# same order

In [45]:
rpkm_pro_s=rpkm_pro.sort_values(by=["gene_id","sample"]).reset_index()
len(rpkm_pro_s)
rpkm_dis_s=rpkm_dis.sort_values(by=["gene_id","sample"]).reset_index()
len(rpkm_dis_s)


736098

736098

# pau

In [46]:
pau=pd.DataFrame(
    data={
        "pau" : rpkm_dis_s["rpkm_dis"] / rpkm_pro_s["rpkm_pro"],
        "sample" : rpkm_dis_s["sample"],
        "gene_id" : rpkm_dis_s["gene_id"]
    }
)
pau.head()

,pau,sample,gene_id
0,0.501881,2026sTSm.Human.Forebrain.CS21.Male,ENSG00000000419.14
1,0.487624,2029sTSb.Human.Hindbrain.CS21.Male,ENSG00000000419.14
2,0.564363,2031sTS.Human.Hindbrain.CS22.Male,ENSG00000000419.14
3,0.350359,2033sTS.Human.Hindbrain.CS22.Female,ENSG00000000419.14
4,0.368783,2034sTS.Human.Forebrain.CS22.Female,ENSG00000000419.14


# filter samples based on fragmentation score 

In [47]:
tmp=pd.merge(metadata,frag_score,on="sample",how='right')
md_fil=tmp[tmp.frag_score>0.885]
md_fil_p=pd.merge(md_fil,period,on="Developmental_Stage")
md_fil_p.head()

,sample,Sex,Developmental_Stage,Age,Unit,organism,frag_score,period,period.abbre,period.abbre.abbre
0,2033sTS.Human.Hindbrain.CS22.Female,female,8 pcw,8,week,hindbrain,0.890121,Early fetal,Fetal,Before birth
1,2035sTSm.Human.Forebrain.8w.Male,male,8 pcw,8,week,forebrain,0.893695,Early fetal,Fetal,Before birth
2,2036sTSm.Human.Hindbrain.8w.Male,male,8 pcw,8,week,hindbrain,0.885023,Early fetal,Fetal,Before birth
3,2080sTS.Human.Brain.CS23.Male,male,8 pcw,8,week,forebrain,0.892380,Early fetal,Fetal,Before birth
4,2082sTSb.Human.Hindbrain.CS23.Female,female,8 pcw,8,week,hindbrain,0.890298,Early fetal,Fetal,Before birth


# merge and filter lowly expressed samples

In [48]:
m1=pd.merge(rpkm_dis_s,rpkm_pro_s,on=["gene_id","sample"])
m2=pd.merge(m1,pau,on=["gene_id","sample"])
final=pd.merge(m2,md_fil_p,on="sample",how='right')
final=final[final.rpkm_pro>1]
len(final)
final.head()

285451

,index_x,type_x,gene_id,rpkm_dis,sample,index_y,type_y,rpkm_pro,pau,Sex,Developmental_Stage,Age,Unit,organism,frag_score,period,period.abbre,period.abbre.abbre
0,45038,ENSG00000000419.14_distal,ENSG00000000419.14,9.654136,2033sTS.Human.Hindbrain.CS22.Female,45039,ENSG00000000419.14_proximal,27.555012,0.350359,female,8 pcw,8,week,hindbrain,0.890121,Early fetal,Fetal,Before birth
1,39629,ENSG00000000457.14_distal,ENSG00000000457.14,2.139920,2033sTS.Human.Hindbrain.CS22.Female,39632,ENSG00000000457.14_proximal,3.837943,0.557570,female,8 pcw,8,week,hindbrain,0.890121,Early fetal,Fetal,Before birth
2,39631,ENSG00000000460.17_distal,ENSG00000000460.17,2.798944,2033sTS.Human.Hindbrain.CS22.Female,39630,ENSG00000000460.17_proximal,2.931922,0.954645,female,8 pcw,8,week,hindbrain,0.890121,Early fetal,Fetal,Before birth
4,47217,ENSG00000001167.15_distal,ENSG00000001167.15,12.118010,2033sTS.Human.Hindbrain.CS22.Female,47216,ENSG00000001167.15_proximal,12.650916,0.957876,female,8 pcw,8,week,hindbrain,0.890121,Early fetal,Fetal,Before birth
5,47816,ENSG00000001630.18_distal,ENSG00000001630.18,42.163486,2033sTS.Human.Hindbrain.CS22.Female,47817,ENSG00000001630.18_proximal,43.390690,0.971717,female,8 pcw,8,week,hindbrain,0.890121,Early fetal,Fetal,Before birth


# wilcox test

In [49]:
sta_res=[]
for i in t_exon_g_l['gene_id'].values:
    pau_b=final[(final.gene_id==i) & (final["period.abbre.abbre"]=="Before birth")].pau
    pau_a=final[(final.gene_id==i) & (final["period.abbre.abbre"]=="After birth")].pau
    sta_res.append([i,ss.mannwhitneyu(pau_b,pau_a,use_continuity=True).pvalue,pau_a.median(),pau_b.median()])

sta_res=pd.DataFrame(sta_res,columns=["gene_id","pvalue","pau_a","pau_b"])
sta_res.head()

,gene_id,pvalue,pau_a,pau_b
0,ENSG00000134317.18,0.001134,1.386422,1.030961
1,ENSG00000106261.17,0.014398,1.619439,1.290808
2,ENSG00000283761.1,0.987734,0.798599,0.731869
3,ENSG00000166997.8,0.626381,1.089880,1.096490
4,ENSG00000214309.5,0.832945,2.416971,2.510632


# write to file

In [50]:
sta_res.to_csv("/home/user/data2/lit/project/ZNF271/02-APA/output/final_res.txt",sep='\t',header=True,index=0)

In [4]:
x=[1.83,  0.50,  1.62,  2.48, 1.68, 1.88, 1.55, 3.06, 1.30]
y=[0.878, 0.647, 0.598, 2.05, 1.06, 1.29, 1.06, 3.14, 1.29]
ss.mannwhitneyu(x,y,use_continuity=True).pvalue

0.13291945818531892

In [21]:
def _mwu_choose_method(n1, n2, xy, method):
    """Choose method 'asymptotic' or 'exact' depending on input size, ties"""

    # if both inputs are large, asymptotic is OK
    if n1 >= 50 or n2 >= 50:
        return "asymptotic"

    # if there are any ties, asymptotic is preferred
    if np.apply_along_axis(_tie_check, -1, xy).any():
        return "asymptotic"

    return "exact"

x=[1.83,  0.50,  1.62,  2.48, 1.68, 1.88, 1.55, 3.06, 1.30]
y=[0.878, 0.647, 0.598, 2.05, 1.06, 1.29, 1.06, 3.14, 1.29]
ss.mannwhitneyu(x,y,use_continuity=True).pvalue

0.13291945818531892